In [0]:
#!!pip uninstall tensorflow
#!pip install tensorflow==2.1.0


# New Section

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
PROJECT_DIR = "/content/drive/My Drive/Colab Notebooks/cleansed_data.csv"
DATA_FILE_NAME = 'My Drive/Colab Notebooks/input_data.xlsx'
CLEANSED_FILE_DIR = '/My Drive/Colab Notebooks/'

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer


import warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
import keras
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, LSTM, Dropout, Embedding , Activation, Bidirectional, Flatten, BatchNormalization
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping


Using TensorFlow backend.


In [0]:
def load_doc(filename):
  if(filename.endswith('.xlsx')):
    data_df = pd.read_excel(filename, lines=True)
  elif(filename.endswith('.csv')):
    data_df = pd.read_csv(filename, keep_default_na = False,delimiter=',')
  return data_df

In [0]:
def encode_target(target):
  le = LabelEncoder()
  return le.fit_transform(target), le

def decode_prediction(pred, encoder):
  return encoder.inverse_transform(pred)

In [71]:
cleansed_data_df = load_doc("/content/drive/My Drive/Colab Notebooks/input_data.xlsx")
print('shape of Data : ', cleansed_data_df.shape)

shape of Data :  (8500, 4)


In [72]:
cleansed_data_df.describe()

,Short description,Description,Caller,Assignment group
count,8492,8499,8500,8500
unique,7481,7817,2950,74
top,password reset,the,bpctwhsn kzqsbmtp,GRP_0
freq,38,56,810,3976


In [73]:
cleansed_data_df.head()

,Short description,Description,Caller,Assignment group
0,login issue,-verified user details.(employee# & manager na...,spxjnwir pjlcoqds,GRP_0
1,outlook,\r\n\r\nreceived from: hmjdrvpb.komuaywn@gmail...,hmjdrvpb komuaywn,GRP_0
2,cant log in to vpn,\r\n\r\nreceived from: eylqgodm.ybqkwiam@gmail...,eylqgodm ybqkwiam,GRP_0
3,unable to access hr_tool page,unable to access hr_tool page,xbkucsvz gcpydteq,GRP_0
4,skype error,skype error,owlgqjme qhcozdfx,GRP_0


In [0]:
cleansed_data_df["Short description"].fillna("The", inplace=True)
cleansed_data_df["Description"].fillna("The", inplace=True)

In [0]:
cleansed_data = cleansed_data_df.copy()

In [82]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [0]:
def remove_punct(text):
    text = re.sub('[0-9]+|\n|\r|[!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]', ' ', text)
    return text

In [0]:
import re
cleansed_data['Description_new'] = cleansed_data['Description'].apply(lambda x: remove_punct(x))
cleansed_data['Short description'] = cleansed_data['Short description'].apply(lambda x: remove_punct(x))

In [0]:
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [0]:
from wordcloud import WordCloud, STOPWORDS

In [90]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [91]:
from nltk.corpus import stopwords
stop_words=set(stopwords.words("english"))
print(stop_words)

{'down', 'all', 'out', 'ma', 'been', 'very', 'where', 'not', "you're", 'ourselves', "weren't", 'against', "you've", "shouldn't", 'same', 're', "wouldn't", "you'll", 'my', 'which', "shan't", 'she', 'her', 'won', 'off', 'these', 'mightn', 'haven', 'again', "doesn't", 'they', "that'll", 'but', "won't", 'those', 'shouldn', 'a', "mightn't", 'who', 'now', 'don', 'll', 'in', 'him', 'that', 'into', 'an', 'needn', 'we', 'the', 'weren', 'during', 'm', 'through', 'didn', "don't", 'it', 'at', 'be', 'over', 'each', 'd', 'you', 'have', 'below', 'any', 'me', 'only', 'mustn', 'our', 'shan', 's', 'too', 'doing', 'what', 'how', 'most', 't', 'isn', "didn't", "couldn't", "hadn't", 'hadn', 'yours', 'did', 'yourself', "hasn't", 'will', 'their', 'were', 'no', 'your', 'just', "it's", 'nor', 'ours', 'themselves', 'by', 'couldn', 'few', 'of', 'is', 'from', 'i', 'while', 'there', 'can', 'such', 'am', 'here', 'both', 'up', 'should', "she's", 'itself', 'had', 'once', 'own', "needn't", 'theirs', 'above', 'between',

In [0]:
cleansed_data['Description_new1'] = cleansed_data['Description_new'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
cleansed_data['Short description'] = cleansed_data['Short description'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

In [94]:
cleansed_data[['Description','Description_new','Description_new1']].head(10)

,Description,Description_new,Description_new1
0,-verified user details.(employee# & manager na...,verified user details employee manager na...,verified user details employee manager name ch...
1,\r\n\r\nreceived from: hmjdrvpb.komuaywn@gmail...,received from hmjdrvpb komuaywn gmail com...,received hmjdrvpb komuaywn gmail com hello tea...
2,\r\n\r\nreceived from: eylqgodm.ybqkwiam@gmail...,received from eylqgodm ybqkwiam gmail com...,received eylqgodm ybqkwiam gmail com hi cannot...
3,unable to access hr_tool page,unable to access hr tool page,unable access hr tool page
4,skype error,skype error,skype error
5,unable to log in to engineering tool and skype,unable to log in to engineering tool and skype,unable log engineering tool skype
6,event: critical:HostName_221.company.com the v...,event critical HostName company com the val...,event critical HostName company com value moun...
7,ticket_no1550391- employment status - new non-...,ticket no employment status new non employ...,ticket employment status new non employee ente...
8,unable to disable add ins on outlook,unable to disable add ins on outlook,unable disable add ins outlook
9,ticket update on inplant_874773,ticket update on inplant,ticket update inplant


In [0]:
cleansed_data['Description'] = cleansed_data['Description_new1']

In [0]:
import re
text1 = "THIS COMMUNICATION IS INTENDED FOR THE SOLE USE OF THE PERSON TO WHOM IT IS ADDRESSED AND MAY CONTAIN INFORMATION THAT IS PRIVILEGED, CONFIDENTIAL AND EXEMPT FROM DISCLOSURE UNDER APPLICABLE LAW. ANY DISSEMINATION, DISTRIBUTION OR DUPLICATION OF THIS COMMUNICATION BY SOMEONE OTHER THAN THE INTENDED RECIPIENT IS STRICTLY PROHIBITED. IF YOUR RECEIPT OF THIS COMMUNICATION IS IN ERROR, PLEASE NOTIFY THE SENDER AND DELETE THIS COMMUNICATION"
cleansed_data['Description'] = cleansed_data['Description'].apply(lambda x: re.sub(text1,'',x))

In [0]:
text2 = "Select the following link to view the Disclaimer in an alternate language."
cleansed_data['Description'] = cleansed_data['Description'].apply(lambda x: re.sub(text2,'',x))

In [0]:
cleansed_data["SD - DD"] = cleansed_data['Short description'].str.cat(cleansed_data["Description"], sep= ' - ')

In [109]:
print(cleansed_data.columns)
cleansed_data.head(10)

Index(['Short description', 'Description', 'Caller', 'Assignment group',
       'Description_new', 'Description_new1', 'SD - DD'],
      dtype='object')


,Short description,Description,Caller,Assignment group,Description_new,Description_new1,SD - DD
0,login issue,verified user details employee manager name ch...,spxjnwir pjlcoqds,GRP_0,verified user details employee manager na...,verified user details employee manager name ch...,login issue - verified user details employee m...
1,outlook,received hmjdrvpb komuaywn gmail com hello tea...,hmjdrvpb komuaywn,GRP_0,received from hmjdrvpb komuaywn gmail com...,received hmjdrvpb komuaywn gmail com hello tea...,outlook - received hmjdrvpb komuaywn gmail com...
2,cant log vpn,received eylqgodm ybqkwiam gmail com hi cannot...,eylqgodm ybqkwiam,GRP_0,received from eylqgodm ybqkwiam gmail com...,received eylqgodm ybqkwiam gmail com hi cannot...,cant log vpn - received eylqgodm ybqkwiam gmai...
3,unable access hr tool page,unable access hr tool page,xbkucsvz gcpydteq,GRP_0,unable to access hr tool page,unable access hr tool page,unable access hr tool page - unable access hr ...
4,skype error,skype error,owlgqjme qhcozdfx,GRP_0,skype error,skype error,skype error - skype error
5,unable log engineering tool skype,unable log engineering tool skype,eflahbxn ltdgrvkz,GRP_0,unable to log in to engineering tool and skype,unable log engineering tool skype,unable log engineering tool skype - unable log...
6,event critical HostName company com value moun...,event critical HostName company com value moun...,jyoqwxhz clhxsoqy,GRP_1,event critical HostName company com the val...,event critical HostName company com value moun...,event critical HostName company com value moun...
7,ticket employment status new non employee ente...,ticket employment status new non employee ente...,eqzibjhw ymebpoih,GRP_0,ticket no employment status new non employ...,ticket employment status new non employee ente...,ticket employment status new non employee ente...
8,unable disable add ins outlook,unable disable add ins outlook,mdbegvct dbvichlg,GRP_0,unable to disable add ins on outlook,unable disable add ins outlook,unable disable add ins outlook - unable disabl...
9,ticket update inplant,ticket update inplant,fumkcsji sarmtlhy,GRP_0,ticket update on inplant,ticket update inplant,ticket update inplant - ticket update inplant


In [0]:
labelencoder = LabelEncoder()
cleansed_data['Assignment group'] = labelencoder.fit_transform(cleansed_data['Assignment group'])

In [0]:
target_count = cleansed_data['Assignment group'].value_counts()

In [219]:
(target_count<=10).value_counts()

False    49
True     25
Name: Assignment group, dtype: int64

In [112]:
oneHotencoder = OneHotEncoder()

#reshape the 1-D country array to 2-D as fit_transform expects 2-D and finally fit the object 
X = oneHotencoder.fit_transform(cleansed_data['Assignment group'].values.reshape(-1,1)).toarray()
print(X)
#print(cleansed_data.shape[1])
#To add this back into the original dataframe 
#dfOneHot = pd.DataFrame(X, columns = ["assignment_group_"+str(int(i)) for i in range(cleansed_data.shape[1])]) 
#df = pd.concat([cleansed_data, dfOneHot], axis=1)
#droping the country column 
#df= df.drop(['assignment_group'], axis=1) 
#printing to verify 
#print(df.head())


[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [0]:
from keras.layers.embeddings import Embedding

from sklearn.model_selection import train_test_split

# Split the data
x_train_keras, x_test_keras, y_train_keras, y_test_keras = train_test_split(cleansed_data['SD - DD'], X, test_size=0.15, shuffle= True)
x_train_keras, x_valid_keras, y_train_keras, y_valid_keras = train_test_split(x_train_keras, y_train_keras, test_size=0.15, shuffle= True)


In [116]:
print(x_train_keras.shape)
print(x_valid_keras.shape)
print(x_test_keras.shape)
print(y_train_keras.shape)
print(y_valid_keras.shape)
print(y_test_keras.shape)

(6141,)
(1084,)
(1275,)
(6141, 74)
(1084, 74)
(1275, 74)


In [0]:
from keras.preprocessing import text
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

In [0]:
max_words = 10000
# finally, vectorize the text samples into a 2D integer tensor
tokenizer = text.Tokenizer(num_words=max_words, char_level=False)
tokenizer.fit_on_texts(x_train_keras)
sequences_train = tokenizer.texts_to_sequences(x_train_keras)
sequences_valid = tokenizer.texts_to_sequences(x_valid_keras)
sequences_test = tokenizer.texts_to_sequences(x_test_keras)

In [122]:
MAX_SEQUENCE_LENGTH = 150

# pad sequences with 0s
X_train = pad_sequences(sequences_train, maxlen=MAX_SEQUENCE_LENGTH)
X_test = pad_sequences(sequences_test, maxlen=MAX_SEQUENCE_LENGTH)
X_valid = pad_sequences(sequences_valid, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X_train.shape)
print('Shape of data test tensor:', X_test.shape)
print('Shape of data validation tensor:', X_valid.shape)

Shape of data tensor: (6141, 150)
Shape of data test tensor: (1275, 150)
Shape of data validation tensor: (1084, 150)


In [128]:
type(x_train_keras)

pandas.core.series.Series

In [131]:
x_train_keras.head()

616     uacyltoe hxgaycze ticket - uacyltoe hxgaycze t...
6632    access requited vl n post goods issues - recei...
6220    problem erp logon - received zaeduhlt jdgsamtv...
5933    msc communicating erp - name tmyeqika hfudpeot...
3252    purchasing menue shopping cart working correct...
Name: SD - DD, dtype: object

In [134]:
X_train[0:6]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,  158,  193,   21,  158, 

In [135]:
print(x_train_keras.iloc[30])
print(X_train[30])
print(x_train_keras.iloc[26])
print(X_train[26])

system frozen startup - system frozen startup
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0   34 2112 1572   34 2112 1572]
erp netweaver enterence please help aerp - received chbvyjqr dqbwijvy gmail com colleagues enter programdntys need immediately please help aerp please see cid image png b b cid image png b b b

In [166]:
# load the whole embedding into memory
embeddings_index = dict()
f = open('../content/drive/My Drive/Colab Notebooks/glove.6B.100d.txt')

for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs

f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 7341 word vectors.


In [169]:
size_of_vocabulary=len(tokenizer.word_index) + 1 #+1 for padding
print(size_of_vocabulary)

12952


In [0]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((size_of_vocabulary, 100))

for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [0]:
from keras.layers import Dense, Input, Flatten, BatchNormalization, Dropout
from keras.layers import GlobalAveragePooling1D, Embedding, LSTM, Bidirectional
from keras.models import Model, Sequential
from keras.callbacks import ReduceLROnPlateau,ModelCheckpoint,EarlyStopping

In [185]:
pip install keras_self_attention

  Created wheel for keras-self-attention: filename=keras_self_attention-0.42.0-cp36-none-any.whl size=17296 sha256=e75dc9c0d894a9081135ee604e24283621d1d64b95977ac6adf0868c82b28f4c
  Stored in directory: /root/.cache/pip/wheels/7b/05/a0/99c0cf60d383f0494e10eca2b238ea98faca9a1fe03cac2894
Successfully built keras-self-attention


In [0]:
from keras_self_attention import SeqSelfAttention

In [0]:
# input: a sequence of MAX_SEQUENCE_LENGTH integers
#EMBEDDING_DIM = 100
#N_CLASSES = 74

#sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
#embedding_layer = Embedding(max_words, EMBEDDING_DIM,
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)
#embedded_sequences = embedding_layer(sequence_input)

#x = Bidirectional(LSTM(128, dropout=0.2, recurrent_dropout=0.2))(embedded_sequences)
#predictions = Dense(N_CLASSES, activation='softmax')(x)

#model = Model(sequence_input, predictions)
#opt = Adam(lr = 0.001)
#model.compile(loss='categorical_crossentropy',
              optimizer='Adam',
              metrics=['acc'])
#Adding callbacks
#es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=3)  
#es = EarlyStopping(monitor='val_loss', min_delta=0.00001, patience=15, verbose=1, mode='auto', baseline=None, restore_best_weights=False)  
#rlr = ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=3, verbose=0, mode="auto", min_delta=0.0001, cooldown=0)
#mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', save_best_only=True,verbose=1)

In [0]:
model=Sequential()
#embedding layer
#model.add(Input(shape=(150,)))
model.add(Embedding(size_of_vocabulary,100,weights=[embedding_matrix],input_length=150,trainable=True)) 

#Bilstm layer
model.add(Bidirectional(LSTM(128,return_sequences=True,dropout=0.2,recurrent_dropout=0.2)))
model.add(SeqSelfAttention(units = 32, attention_activation='relu'))

#Global Maxpooling
model.add(Flatten())

#Dense Layer
#model.add(Dense(256,activation='relu'))  
#model.add(Dense(128,activation='relu'))  
model.add(Dense(74,activation='softmax')) 

In [0]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])

#Adding callbacks
#es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=3)  
es = EarlyStopping(monitor='val_loss', min_delta=0.00001, patience=15, verbose=1, mode='auto', baseline=None, restore_best_weights=False)  
rlr = ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=3, verbose=0, mode="auto", min_delta=0.0001, cooldown=0)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', save_best_only=True,verbose=1)

In [197]:
model.summary()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_29 (Embedding)     (None, 150, 100)          1295200   
_________________________________________________________________
bidirectional_21 (Bidirectio (None, 150, 256)          234496    
_________________________________________________________________
seq_self_attention_2 (SeqSel (None, 150, 256)          16449     
_________________________________________________________________
flatten_11 (Flatten)         (None, 38400)             0         
_________________________________________________________________
dense_24 (Dense)             (None, 74)                2841674   
Total params: 4,387,819
Trainable params: 4,387,819
Non-trainable params: 0
_________________________________________________________________


In [198]:
historical = model.fit(X_train,y_train_keras,batch_size=32,epochs=50,validation_data=(X_valid,y_valid_keras),verbose=1,callbacks=[es,rlr,mc])

Train on 6141 samples, validate on 1084 samples
Epoch 1/50
6141/6141 [==============================] - 122s 20ms/step - loss: 2.1922 - acc: 0.5261 - val_loss: 1.8028 - val_acc: 0.5839

Epoch 00001: val_acc improved from -inf to 0.58395, saving model to best_model.h5
Epoch 2/50
6141/6141 [==============================] - 121s 20ms/step - loss: 1.6015 - acc: 0.6061 - val_loss: 1.6445 - val_acc: 0.6061

Epoch 00002: val_acc improved from 0.58395 to 0.60609, saving model to best_model.h5
Epoch 3/50
6141/6141 [==============================] - 121s 20ms/step - loss: 1.1743 - acc: 0.6794 - val_loss: 1.4719 - val_acc: 0.6513

Epoch 00003: val_acc improved from 0.60609 to 0.65129, saving model to best_model.h5
Epoch 4/50
6141/6141 [==============================] - 116s 19ms/step - loss: 0.8172 - acc: 0.7616 - val_loss: 1.4713 - val_acc: 0.6753

Epoch 00004: val_acc improved from 0.65129 to 0.67528, saving model to best_model.h5
Epoch 5/50
6141/6141 [==============================] - 115s 19

In [143]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_15 (Embedding)     (None, 150, 100)          1000000   
_________________________________________________________________
bidirectional_7 (Bidirection (None, 150, 256)          234496    
_________________________________________________________________
dense_9 (Dense)              (None, 150, 128)          32896     
_________________________________________________________________
dense_10 (Dense)             (None, 150, 52)           6708      
Total params: 1,274,100
Trainable params: 1,274,100
Non-trainable params: 0
_________________________________________________________________


In [48]:
history = model.fit(X_train, y_train_keras,
          nb_epoch=10, batch_size=32)

Epoch 1/10
4877/4877 [==============================] - 29s 6ms/step - loss: 2.6111 - acc: 0.4831
Epoch 2/10
4877/4877 [==============================] - 29s 6ms/step - loss: 2.3042 - acc: 0.4995
Epoch 3/10
4877/4877 [==============================] - 29s 6ms/step - loss: 2.1151 - acc: 0.5194
Epoch 4/10
4877/4877 [==============================] - 29s 6ms/step - loss: 1.9403 - acc: 0.5434
Epoch 5/10
4877/4877 [==============================] - 29s 6ms/step - loss: 1.7509 - acc: 0.5780
Epoch 6/10
4877/4877 [==============================] - 29s 6ms/step - loss: 1.5949 - acc: 0.5989
Epoch 7/10
4877/4877 [==============================] - 29s 6ms/step - loss: 1.4494 - acc: 0.6334
Epoch 8/10
4877/4877 [==============================] - 29s 6ms/step - loss: 1.3151 - acc: 0.6598
Epoch 9/10
4877/4877 [==============================] - 29s 6ms/step - loss: 1.2237 - acc: 0.6779
Epoch 10/10
4877/4877 [==============================] - 29s 6ms/step - loss: 1.0958 - acc: 0.7119


In [0]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

y_train_cate = to_categorical(y_train_keras)
y_test_cate = to_categorical(y_valid_keras)
print(y_train_cate.shape)

x_train_keras = x_train_keras.astype(str)
x_valid_keras = x_valid_keras.astype(str)

#Tokenize the sentences
tokenizer = Tokenizer(num_words=10000,char_level=False)

#preparing vocabulary
tokenizer.fit_on_texts(list(x_train_keras))

#converting text into integer sequences
x_tr_seq_keras  = tokenizer.texts_to_sequences(x_train_keras) 
x_val_seq_keras = tokenizer.texts_to_sequences(x_valid_keras)

#padding to prepare sequences of same length
x_tr_seq_keras  = pad_sequences(x_tr_seq_keras, maxlen=100,padding='post')
x_val_seq_keras = pad_sequences(x_val_seq_keras, maxlen=100,padding='post')

size_of_vocabulary=len(tokenizer.word_index) + 1 #+1 for padding
print(size_of_vocabulary)

(4335, 52, 2)
2872


In [0]:
print(x_train_keras[24])
print(x_tr_seq_keras[24])
print(x_train_keras[52])
print(x_tr_seq_keras[52])
print(x_tr_seq_keras[24])
print(x_tr_seq_keras[25])

password reset password reset
[148   5 858   5 126 405  11   9 248 109   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0]
outlook received
[159 227   6 159 227   6   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0]
[148   5 858   5 126 405  11   9 248 109   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0  

In [139]:
#from keras.layers.embeddings import Embedding
#from keras.optimizers import Adam


model=Sequential()
#embedding layer
model.add(Embedding(10000,100,input_length=100,trainable=True)) 

#lstm layer
model.add(Bidirectional(LSTM(128,return_sequences=True,dropout=0.2)))

#Global Maxpooling
model.add(Flatten())

#Dense Layer
#model.add(Dense(256,activation='relu'))  
model.add(Dense(128,activation='relu'))  
model.add(Dense(52,activation='softmax'))  

#Add loss function, metrics, optimizer
model.compile(optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-15, decay=0.0), loss='categorical_crossentropy',metrics=["acc"]) 

#Adding callbacks
#es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=3)  
es = EarlyStopping(monitor='val_loss', min_delta=0.00001, patience=15, verbose=1, mode='auto', baseline=None, restore_best_weights=False)  

mc=ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', save_best_only=True,verbose=1)  

#Print summary of model
print(model.summary())

TypeError: ignored

In [0]:
model.fit(x_tr_seq_keras,y_train_keras,batch_size=32,epochs=100,validation_data=(x_val_seq_keras,y_valid_keras),verbose=1,callbacks=[es,mc])
#model.fit(x_tr_seq_fast,y_train_fast,batch_size=32,epochs=10,validation_data=(x_val_seq_fast,y_valid_fast),verbose=1,callbacks=[es,mc])

Epoch 1/100
136/136 [==============================] - ETA: 0s - loss: 2.4157 - acc: 0.4960
Epoch 00001: val_acc improved from -inf to 0.48339, saving model to best_model.h5
136/136 [==============================] - 36s 266ms/step - loss: 2.4157 - acc: 0.4960 - val_loss: 2.3297 - val_acc: 0.4834
Epoch 2/100
136/136 [==============================] - ETA: 0s - loss: 1.9713 - acc: 0.5412
Epoch 00002: val_acc improved from 0.48339 to 0.50830, saving model to best_model.h5
136/136 [==============================] - 40s 294ms/step - loss: 1.9713 - acc: 0.5412 - val_loss: 2.1687 - val_acc: 0.5083
Epoch 3/100
136/136 [==============================] - ETA: 0s - loss: 1.5999 - acc: 0.5991
Epoch 00003: val_acc improved from 0.50830 to 0.52214, saving model to best_model.h5
136/136 [==============================] - 36s 262ms/step - loss: 1.5999 - acc: 0.5991 - val_loss: 2.0375 - val_acc: 0.5221
Epoch 4/100
136/136 [==============================] - ETA: 0s - loss: 1.2820 - acc: 0.6713
Epoch 00

In [0]:
#loading best model
from tensorflow.keras.models import load_model
model = load_model('best_model.h5')

#evaluation 
_,val_acc = model.evaluate(x_val_seq_keras,y_valid_keras, batch_size=100)
print(val_acc)

11/11 [==============================] - 2s 158ms/step - loss: 2.0375 - acc: 0.5221
0.5221402049064636


In [0]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

y_train_cate = to_categorical(y_train_keras)
y_test_cate = to_categorical(y_valid_keras)
print(y_train_cate.shape)

x_train_keras = x_train_keras.astype(str)
x_valid_keras = x_valid_keras.astype(str)

#Tokenize the sentences
tokenizer = Tokenizer(num_words=10000,char_level=False)

#preparing vocabulary
tokenizer.fit_on_texts(list(x_train_keras))

#converting text into integer sequences
x_tr_seq_keras  = tokenizer.texts_to_sequences(x_train_keras) 
x_val_seq_keras = tokenizer.texts_to_sequences(x_valid_keras)

#padding to prepare sequences of same length
x_tr_seq_keras_pre  = pad_sequences(x_tr_seq_keras, maxlen=100)
x_val_seq_keras_pre = pad_sequences(x_val_seq_keras, maxlen=100)

size_of_vocabulary=len(tokenizer.word_index) + 1 #+1 for padding
print(size_of_vocabulary)

(4335, 52, 2)
2872


In [0]:
print(x_train_keras[24])
print(x_tr_seq_keras_pre[24])
print(x_train_keras[52])
print(x_tr_seq_keras_pre[52])
print(x_tr_seq_keras_pre[24])
print(x_tr_seq_keras_pre[25])

password reset password reset
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
 148   5 858   5 126 405  11   9 248 109]
outlook received
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0 159 227   6 159 227   6]
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0  

In [0]:
#from keras.layers.embeddings import Embedding
#from keras.optimizers import Adam


model=Sequential()
#embedding layer
model.add(Embedding(size_of_vocabulary,100,input_length=100,trainable=True)) 

#lstm layer
model.add(Bidirectional(LSTM(128,return_sequences=True,dropout=0.2)))

#Global Maxpooling
model.add(Flatten())

#Dense Layer
#model.add(Dense(256,activation='relu'))  
model.add(Dense(128,activation='relu'))  
model.add(Dense(52,activation='softmax'))  

#Add loss function, metrics, optimizer
model.compile(optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-15, decay=0.0), loss='categorical_crossentropy',metrics=["acc"]) 

#Adding callbacks
#es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=3)  
es = EarlyStopping(monitor='val_loss', min_delta=0.00001, patience=15, verbose=1, mode='auto', baseline=None, restore_best_weights=False)  

mc=ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', save_best_only=True,verbose=1)  

#Print summary of model
print(model.summary())

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 100, 100)          287200    
_________________________________________________________________
bidirectional_6 (Bidirection (None, 100, 256)          234496    
_________________________________________________________________
flatten_6 (Flatten)          (None, 25600)             0         
_________________________________________________________________
dense_12 (Dense)             (None, 128)               3276928   
_________________________________________________________________
dense_13 (Dense)             (None, 52)                6708      
Total params: 3,805,332
Trainable params: 3,805,332
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
model.fit(x_tr_seq_keras_pre,y_train_keras,batch_size=32,epochs=10,validation_data=(x_val_seq_keras_pre,y_valid_keras),verbose=1,callbacks=[es,mc])

Epoch 1/10
136/136 [==============================] - ETA: 0s - loss: 0.7309 - acc: 0.8030
Epoch 00001: val_acc improved from 0.52583 to 0.53321, saving model to best_model.h5
136/136 [==============================] - 36s 264ms/step - loss: 0.7309 - acc: 0.8030 - val_loss: 2.6639 - val_acc: 0.5332
Epoch 2/10
136/136 [==============================] - ETA: 0s - loss: 0.5690 - acc: 0.8464
Epoch 00002: val_acc did not improve from 0.53321
136/136 [==============================] - 36s 264ms/step - loss: 0.5690 - acc: 0.8464 - val_loss: 2.7841 - val_acc: 0.5212
Epoch 3/10
136/136 [==============================] - ETA: 0s - loss: 0.4614 - acc: 0.8764
Epoch 00003: val_acc did not improve from 0.53321
136/136 [==============================] - 36s 263ms/step - loss: 0.4614 - acc: 0.8764 - val_loss: 3.0761 - val_acc: 0.5129
Epoch 4/10
136/136 [==============================] - ETA: 0s - loss: 0.3692 - acc: 0.9020
Epoch 00004: val_acc did not improve from 0.53321
136/136 [====================

In [0]:
#loading best model
from tensorflow.keras.models import load_model
model = load_model('best_model.h5')

#evaluation 
_,val_acc = model.evaluate(x_val_seq_keras_pre,y_valid_keras, batch_size=100)
print(val_acc)

11/11 [==============================] - 2s 157ms/step - loss: 2.6639 - acc: 0.5332
0.5332103371620178
